<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd

# 전국일별강수량
강수량 = pd.read_csv('./data/전국일별강수량.csv', encoding='cp949')
강수량 = 강수량[['일시', '평균일강수량(mm)']].dropna(axis=0)
print(강수량.shape)
강수량.head(3)

(730, 2)


,일시,평균일강수량(mm)
0,2019-12-31,0.1
1,2019-12-30,0.9
2,2019-12-29,6.2


In [3]:
# 전국일별기온
기온 = pd.read_csv('./data/전국일별기온.csv', encoding='cp949')
기온 = 기온[['일시', '평균기온(℃)']].dropna(axis=0)
print(기온.shape)
기온.head(3)

(730, 2)


,일시,평균기온(℃)
0,2019-12-31,-3.8
1,2019-12-30,5.0
2,2019-12-29,2.6


In [5]:
# 전국일별습도
습도 = pd.read_csv('./data/전국일별습도.csv', encoding='cp949')
습도 = 습도[['일시', '평균습도(%rh)']].dropna(axis=0)
print(습도.shape)
습도.head(3)

(730, 2)


,일시,평균습도(%rh)
0,2019-12-31,46.0
1,2019-12-30,79.0
2,2019-12-29,73.0


In [8]:
# 전국일별일조일사
일조일사 = pd.read_csv('./data/전국일별일조일사.csv', encoding='cp949')
일조일사 = 일조일사[['일시', '일조합', '일사합']].dropna(axis=0)
print(일조일사.shape)
일조일사.head(3)

(730, 3)


,일시,일조합,일사합
0,2019-12-31,8.8,11.31
1,2019-12-30,2.2,3.96
2,2019-12-29,0.2,3.33


일조 : 태양광선이 다른 물체에 가려지지 않고 직접 지상 물체에 닿는 것

일사 : 지표면에 도달한 태양복사 에너지

In [10]:
# 전국일별풍속
풍속 = pd.read_csv('./data/전국일별풍속.csv', encoding='cp949')
풍속 = 풍속[['일시', '평균풍속(m/s)']].dropna(axis=0)
print(풍속.shape)
풍속.head(3)

(730, 2)


,일시,평균풍속(m/s)
0,2019-12-31,3.6
1,2019-12-30,2.1
2,2019-12-29,1.2


In [25]:
# 1차 데이터 병합
data = pd.concat([기온, 강수량['평균일강수량(mm)'], 풍속['평균풍속(m/s)'], 습도['평균습도(%rh)'], 일조일사[['일조합','일사합']]], axis=1)
print(data.shape)
data.head(3)

(730, 7)


,일시,평균기온(℃),평균일강수량(mm),평균풍속(m/s),평균습도(%rh),일조합,일사합
0,2019-12-31,-3.8,0.1,3.6,46.0,8.8,11.31
1,2019-12-30,5.0,0.9,2.1,79.0,2.2,3.96
2,2019-12-29,2.6,6.2,1.2,73.0,0.2,3.33


**체감온도 = 13.12 + 0.6215T - 11.37 V0.16 + 0.3965 V0.16T**

T : 기온(℃), V : 풍속(km/h) 이므로, 평균풍속(m/s) 칼럼을 km/h로 변환하기 위해 3.6을 곱함

In [26]:
data['평균풍속(m/s)'] = data['평균풍속(m/s)'].apply(lambda x: x*3.6)
data.rename(columns={'평균풍속(m/s)': '평균풍속(km/h)'}, inplace=True)

In [27]:
data.head(3)

,일시,평균기온(℃),평균일강수량(mm),평균풍속(km/h),평균습도(%rh),일조합,일사합
0,2019-12-31,-3.8,0.1,12.96,46.0,8.8,11.31
1,2019-12-30,5.0,0.9,7.56,79.0,2.2,3.96
2,2019-12-29,2.6,6.2,4.32,73.0,0.2,3.33


In [29]:
# 전국일별체감온도 계산
def temp(T, V):
    return 13.12 + 0.6215*T - 11.37*(V**0.16) + 0.3965*(V**0.16)*T

data['체감온도(℃)'] = data.apply(lambda x: temp(x['평균기온(℃)'], x['평균풍속(km/h)']), axis=1)
data.head(3)

,일시,평균기온(℃),평균일강수량(mm),평균풍속(km/h),평균습도(%rh),일조합,일사합,체감온도(℃)
0,2019-12-31,-3.8,0.1,12.96,46.0,8.8,11.31,-8.642578
1,2019-12-30,5.0,0.9,7.56,79.0,2.2,3.96,3.252319
2,2019-12-29,2.6,6.2,4.32,73.0,0.2,3.33,1.669374


In [30]:
# 전국미세먼지농도
미세먼지 = pd.read_csv('./data/전국미세먼지농도.csv', encoding='cp949')
print(미세먼지.shape)
미세먼지.head(3)

(19010, 4)


,지점,지점명,일시,일 미세먼지 농도(㎍/㎥)
0,90,속초,2018-01-01,18
1,90,속초,2018-01-02,23
2,90,속초,2018-01-03,19


In [31]:
미세먼지 = pd.DataFrame(미세먼지.groupby('일시')['일 미세먼지 농도(㎍/㎥)'].mean())
print(미세먼지.shape)
미세먼지.head(3)

(728, 1)


,일 미세먼지 농도(㎍/㎥)
일시,
2018-01-01,32.962963
2018-01-02,40.037037
2018-01-03,23.185185


In [52]:
# 2차 데이터 병합
data2 = pd.merge(data, 미세먼지, on='일시', how='left')
data2.tail(3)

,일시,평균기온(℃),평균일강수량(mm),평균풍속(km/h),평균습도(%rh),일조합,일사합,체감온도(℃),일 미세먼지 농도(㎍/㎥)
727,2018-01-03,-1.6,0.0,8.64,46.0,8.6,9.35,-4.824906,23.185185
728,2018-01-02,0.2,0.0,6.48,57.0,6.6,6.89,-1.981246,40.037037
729,2018-01-01,-0.4,0.0,6.84,51.0,8.7,9.31,-2.810026,32.962963


In [54]:
data2[data2['일 미세먼지 농도(㎍/㎥)'].isnull()]

,일시,평균기온(℃),평균일강수량(mm),평균풍속(km/h),평균습도(%rh),일조합,일사합,체감온도(℃),일 미세먼지 농도(㎍/㎥)
83,2019-10-09,13.3,0.0,4.68,64.0,10.4,18.51,13.581838,NaN
84,2019-10-08,15.9,0.1,7.92,67.0,9.0,16.31,15.947919,NaN


In [56]:
data2.to_csv('./data/전국날씨데이터.csv', index=False)